# BiLSTM-CRF (helper-based)
This notebook shows how to build, train, and evaluate a BiLSTM-CRF using keras_crf.train_utils helpers.

In [ ]:
import os
os.environ.setdefault('KERAS_BACKEND', os.getenv('KERAS_BACKEND', 'tensorflow'))
import numpy as np
import keras
from keras import layers
import sys, pathlib
root = pathlib.Path(__file__).resolve().parents[2]
sys.path.insert(0, str(root))
from examples.utils.data import make_varlen_dataset
from examples.utils.metrics import MaskedTokenAccuracy
from keras_crf.train_utils import make_crf_tagger, prepare_crf_targets


In [ ]:
# Synthetic dataset
X_train, Y_train, _ = make_varlen_dataset(1000, 40, 200, 6, seed=1)
X_val, Y_val, _ = make_varlen_dataset(200, 40, 200, 6, seed=2)
vocab_size = 200
num_tags = 6


In [ ]:
tokens_in = keras.Input(shape=(None,), dtype='int32', name='tokens')
x = layers.Embedding(vocab_size + 1, 64, mask_zero=True)(tokens_in)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
model = make_crf_tagger(tokens_in, x, num_tags, metrics=[MaskedTokenAccuracy()], loss='dice+nll', joint_nll_weight=0.2)
y_train, sw_train = prepare_crf_targets(Y_train, mask=(X_train != 0).astype(np.float32))
y_val, sw_val = prepare_crf_targets(Y_val, mask=(X_val != 0).astype(np.float32))
model.fit({'tokens': X_train, 'labels': Y_train}, y_train, sample_weight=sw_train,
          validation_data=({'tokens': X_val, 'labels': Y_val}, y_val, sw_val),
          epochs=2, batch_size=64, verbose=2)
